# Stocks

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import ig

## API Setup

In [2]:
details = ig.get_api('prod')
api = ig.APIHandler(details['url'], details['api_key'], details['user_name'], details['password'])
del details

In [4]:
# api.markets(93334)
print(api.prices('IX.D.FTSE.DAILY.IP', resolution='HOUR_4',
                 from_date='2020-08-13 09:00:00', to_date='2020-08-14 11:00:00'))

https://api.ig.com/gateway/deal/prices/IX.D.FTSE.DAILY.IP/HOUR_4/2020-08-13%2009%3A00%3A00/2020-08-14%2011%3A00%3A00
[{'prices': [{'snapshotTime': '2020/08/13 12:00:00', 'openPrice': {'bid': 6207.5, 'ask': 6208.5, 'lastTraded': None}, 'closePrice': {'bid': 6191.6, 'ask': 6192.6, 'lastTraded': None}, 'highPrice': {'bid': 6219.5, 'ask': 6220.5, 'lastTraded': None}, 'lowPrice': {'bid': 6187.6, 'ask': 6188.6, 'lastTraded': None}, 'lastTradedVolume': 27359}, {'snapshotTime': '2020/08/13 16:00:00', 'openPrice': {'bid': 6191.8, 'ask': 6192.8, 'lastTraded': None}, 'closePrice': {'bid': 6167.3, 'ask': 6169.3, 'lastTraded': None}, 'highPrice': {'bid': 6198.3, 'ask': 6199.6, 'lastTraded': None}, 'lowPrice': {'bid': 6160.3, 'ask': 6162.3, 'lastTraded': None}, 'lastTradedVolume': 18937}, {'snapshotTime': '2020/08/13 20:00:00', 'openPrice': {'bid': 6167.6, 'ask': 6169.6, 'lastTraded': None}, 'closePrice': {'bid': 6189.0, 'ask': 6193.0, 'lastTraded': None}, 'highPrice': {'bid': 6189.5, 'ask': 6193.5,